# Privacy - target model

reference : https://keras.io/examples/mnist_cnn/

## Imports

In [ ]:
from __future__ import print_function

%load_ext autoreload
%autoreload 2

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
import os
import matplotlib.pyplot as plt
os.sys.path.append("./src")
from utils import plot_img
from utils import load_mnist_data

## Data and settings

In [ ]:
batch_size = 64
epochs = 4
img_rows, img_cols = 28, 28
num_classes = 10

In [ ]:
x_train, y_train, x_test, y_test = load_mnist_data("all")

### Plot some sample to check if we have only digits from 0 to 4

In [ ]:
plot_img(x_train, y_train)

## Train model

In [ ]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Evaluate the results

In [ ]:
if not "model" in os.listdir("./"):
    os.mkdir("model")
model.save("model/evaluation_model.h5")

In [ ]:
def expand_first_last_dims(array):
    return np.expand_dims(np.expand_dims(array,-1),0)

In [ ]:
results = []
for file in os.listdir("./results"):
    file_path = os.path.join("./results", file)
    result = np.load(file_path, allow_pickle=True)
    label = result[()]["label_targetted"]
    label_detected = model.predict(expand_first_last_dims(result[()]["image_reconstructed"]))
    results.append(np.argmax(label)==np.argmax(label_detected))